IMPORT modules(This part is dataAnalysis.py)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import *


DATA EXTRACTION, ANALYSIS AND SIMPLIFICATION

In [ ]:
features = [
    "previous_loan_defaults_on_file",
    "person_gemder"
]
data = pd.read_csv("loan_data.csv")




data.drop(columns=["loan_int_rate","person_gender"],axis=1,inplace=True)


'''

data.replace({
    "Yes" : 1,
    "No" : 0,
    "male" : 1,
    "female" : 0
},inplace=True)

# print(chi2(data[features],data['loan_status'])) 

output for previous_loan_defaults_on_file : (array([6530.85714286]), array([0.]))
Strong relation

output for person_gender : (array([0.00635183]), array([0.93647718]))
No significant relation

'''
# output shows a very strong and significant relationship


# print(data['previous_loan_defaults_on_file'].value_counts())

# Simplified (basically)
# data.to_csv("loan_dataS.csv",index=False)


''' 
    The point biseral coefficient is a method(score) used to determine
    correltaion between a continuous feture and a categorical(binary) feature
'''
def pointBiseralCoeff(x0,x1):
    m0 = x0.mean()
    m1 = x1.mean()
    s = np.concatenate([x0,x1]).std()
    n0 = len(x0)
    n1 = len(x1)
    n = n0 + n1

    return (((m1 - m0 )/s) * np.sqrt(n0*n1/(n**2)))


data0 = data[data['loan_status']==0]
data1 = data[data['loan_status']==1]

X0 = data0["cb_person_cred_hist_length"]
X1 = data1["cb_person_cred_hist_length"]
print(pointBiseralCoeff(X0,X1)) 


'''
# X0 = data0['loan_percent_income']
# X0 = data0['credit_score']
# X0 = data0["person_income"]
X0 = data0["person_emp_exp"]
X0 = data0["cb_person_cred_hist_length"]
X0 = data0["loan_amnt"]

# X1 = data1['loan_percent_income']
# X1 = data1['credit_score']
# X1 = data1["person_income"]
X1 = data1["person_emp_exp"]
X1 = data1["cb_person_cred_hist_length"]
X1 = data1["loan_amnt"]


print(pointBiseralCoeff(X0,X1)) 

output : 0.3848803799720417 for "loan_percent_income"-> Shows a MODERATELY STRONG RELATIONSHIP
output : -0.007647176334884879 for "credit_score"
output : -0.13580771683574247 for "person_income"-> Shows a WEAK NEGATIVE RELATIONSHIP
output : -0.02048125886337787 for "person_emp_exp"-> Shows a VERY WEAK NEGATIVE RELATIONSHIP
output : -0.014850683660985917 for "cb_person_cred_hist_length"-> Shows a VERY WEAK RELATIONSHIP
output : 0.10771446698132857 for "loan_amnt"-> Shows a  WEAK RELATIONSHIP

'''

MOdel TRAINING AND TESTING ( Content below This was kept in modelTraining.py)

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.utils.class_weight import compute_class_weight
data = pd.read_csv("loan_dataS.csv")
# data = pd.read_csv("loan_dataUnderSmapled.csv")
X = data[data.columns[:-1]].to_numpy()
y = data[data.columns[-1]].to_numpy()

# features = [
#     'person_education',
#     'person_home_ownership',
#     'loan_intent',
#     'previous_loan_defaults_on_file'    
# ]

cc= [1,4,6,10]
ff = [0,2,3,5,7,8,9]
X = pd.DataFrame(X)

for col in cc:
    X[col] = X[col].astype('category')
for col in ff:
    X[col] = X[col].astype('float')


xTrain,xTest,yTrain,yTest = train_test_split(
                                                X, 
                                                 y, 
                                                 test_size=0.22,
                                                random_state=42,
                                                stratify=y
                                            )

# print(np.unique(yTrain))




cWeights = compute_class_weight(class_weight='balanced',classes=np.unique(yTrain),y=yTrain)
sample_weights = np.array([cWeights[0] if y==0 else cWeights[1] for y in yTrain])
print(cWeights)
xgbModel = xgb.XGBClassifier(scale_pos_weight = cWeights[0]/cWeights[1], enable_categorical=True)
# xgbModel = xgb.XGBClassifier(scale_pos_weight = 0.30, enable_categorical=True)
# xgbModel.fit(xTrain, yTrain, sample_weight = sample_weights)
xgbModel.fit(xTrain, yTrain,sample_weight = sample_weights )
y_pred = xgbModel.predict(xTest)
print(classification_report(yTest, y_pred))
print("Accuracy : ",accuracy_score(yTest, y_pred))
print("roc auc score : ",roc_auc_score(yTest, y_pred,multi_class='ovr', average='macro'))


'''
o/p :- 

[0.64285714 2.25      ]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      7700
           1       0.77      0.78      0.77      2200

    accuracy                           0.90      9900
   macro avg       0.85      0.86      0.85      9900
weighted avg       0.90      0.90      0.90      9900

Accuracy :  0.8981818181818182
roc auc score :  0.8556493506493508



'''